In [ ]:
!pip install trackintel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.3/401.3 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.1/156.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 99.3 MB/s eta 0:00:00
  Created wheel for similaritymeasures: filename=similaritymeasures-1.2.0-py3-none-any.whl size=15433 sha256=1e6938b5296141ce43feb88693b65d9d8207c7ef5f695d209b34da7931fabd1b
  Stored in directory: /root/.cache/pip/wheels/0f/8b/4f/b9dd125a4fa600b2d9ee19e728feb3f24bf19bdfd9eeda9191
Successfully built similaritymeasures
  Att

In [ ]:
import pandas as pd
import trackintel as ti
import gzip
import os
import geopandas as gpd


In [ ]:
# Define the path to the .gz files
data_files = {
    'A': "./cityA_groundtruthdata.csv.gz",
    'B': "./cityB_challengedata.csv.gz",
    'C': "./cityC_challengedata.csv.gz",
    'D': "./cityD_challengedata.csv.gz"
}


In [ ]:
# Function to load .gz files into DataFrame
def load_gz_file(file_path):
    with gzip.open(file_path, 'rt') as f:
        return pd.read_csv(f)


In [ ]:
# Function to add 'tracked_at' column, rename uid to user_id, x to longtitude , y to latitude, and drop rows with -999 coordinates
def preprocess_data(city):
    #load data
    df = load_gz_file(data_files[city])

    # Drop rows with missing data (x, y marked as -999)
    df = df[(df['x'] != -999) & (df['y'] != -999)]

    # Add the 'tracked_at' column by combining 'd' and 't' columns
    df['date'] = pd.to_datetime(df['d'], format='%j', errors='coerce')
    df['time'] = pd.to_timedelta(df['t'] * 30, unit='m')
    df['tracked_at'] = df['date'] + df['time']
    df['tracked_at'] = df['tracked_at'].dt.tz_localize('UTC')

    # Drop the intermediate 'date' and 'time' columns
    df.drop(columns=['date', 'time'], inplace=True)

    # Rename columns
    df.rename(columns={'uid': 'user_id', 'x': 'longitude', 'y': 'latitude'}, inplace=True)
    df.to_csv(f'data_{city}_preprocessed.csv', index=False)
    print(f"Data saved to csv")

In [ ]:
def custom_write_triplegs_csv(triplegs, filename, **kwargs):
    """Saves triplegs data to a CSV file with custom options.

    Args:
        triplegs: The triplegs data.
        filename: The name of the output CSV file.
        **kwargs: Additional keyword arguments to pass to to_csv.
    """
    triplegs_df = triplegs.to_wkt(rounding_precision=-1, trim=False)
    triplegs_df.to_csv(filename, **kwargs)  # Use kwargs to control index etc.



In [ ]:
def gen_triplegs(city):
    #preprocess data and save as csv
    print(f"Preprocessing city {city} data")
    preprocess_data(city)

    # Load preprocessed data and generate positionfixes
    print(f"Loading preprocessed data: city {city}")
    pfs = ti.read_positionfixes_csv(f'data_{city}_preprocessed.csv')

    #gerenate staypoints
    print("Generating staypoints")
    pfs , sp = pfs.as_positionfixes.generate_staypoints(
    method='sliding',
    dist_threshold=1,  # Distance threshold in meters (1 cell = 500 meters)
    time_threshold=90,  # Time threshold  (minutes) #test with different values 
    gap_threshold=300,  # Gap threshold  (minutes)  #test with different values 
    distance_metric='haversine',  # Distance metric
    include_last=True,  # Include the last staypoint
    print_progress=True,  # Print progress
    exclude_duplicate_pfs=True,  # Exclude duplicate position fixes
    n_jobs=-1  # Number of jobs for parallel processing
)

    #generate triplegs using positionfixes and staypoints generated earlier
    print("Generating triplegs")
    
    #test with different values
    #larger gap leads to lesser and more continuous trip
    #gap threshold in generate_triplegs should be configured in relation to both time and the gap threshold used in generate_staypoints
    pfs , tpls = ti.preprocessing.generate_triplegs(pfs , sp , method='between_staypoints' , gap_threshold=90) 
    print("Saving Triplegs to csv")
    custom_write_triplegs_csv(tpls, f'triplegs{city}.csv', index=False)
    print("Triplegs saved to csv")




'\ntime_threshold: This is the minimum amount of time a person needs to stay within a small area (defined by dist_threshold) for that area to be considered a staypoint. If the time spent in that spot meets or exceeds time_threshold, it qualifies as a staypoint.\n\ngap_threshold: This is the maximum time allowed between consecutive position fixes for them to be considered part of a continuous sequence. If the time gap between two position fixes is larger than gap_threshold, they are considered discontinuous, meaning they won’t be grouped into the same staypoint.\n'

In [ ]:
gen_triplegs('A')

Preprocessing city A data
Data saved to csv
Loading preprocessed data: city A


/usr/local/lib/python3.10/dist-packages/trackintel/io/util.py:13: UserWarning: Assuming default index as unique identifier. Pass 'index_col=None' as explicit argument to avoid a warning.
  warnings.warn(


Generating staypoints


100%|██████████| 100000/100000 [29:49<00:00, 55.89it/s]


Generating triplegs


/usr/local/lib/python3.10/dist-packages/trackintel/preprocessing/positionfixes.py:361: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pfs["tripleg_id"] = pfs["tripleg_id"].ffill()
/usr/local/lib/python3.10/dist-packages/trackintel/preprocessing/positionfixes.py:573: UserWarning: The positionfixes with ids [   639842    638957    678518 ... 111528316 111528317 111528318] lead to invalid tripleg geometries. The resulting triplegs were omitted and the tripleg id of the positionfixes was set to nan
  warnings.warn(warn_string)


Saving Triplegs to csv
Triplegs saved to csv
